# Proyecto Integrador 01 - Steam 

<br>

## Se realiza proceso de ETL sobre dataset steam_games

##### Importamos las librerías

In [1]:
import pandas as pd
import gzip
import json
import re


<br>
<br>

#### Se extraen los datos del dataset provisto, descomprimiéndolo primero

In [2]:
# Descomprime el archivo JSON.gz
with gzip.open(r'C:\Users\Cebol\OneDrive\Escritorio\Datasets\Datasets\steam_games.json.gz', 'rb') as g:
    # Usa un lector de línea para leer el archivo línea por línea
    reader = g.readlines()

# Se lee de cada línea del dataset
filas_games = []
for line in reader:
    # Decodifica la línea como JSON
    data = json.loads(line.decode('utf-8'))
    # Agrega la línea al dataframe
    filas_games.append(data)

# Se convierte en dataframe
df_g = pd.DataFrame(filas_games)
df_g


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


<br>

Como primera medida, se observa una gran cantidad de filas, que contienen datos Nan en todos los campos. Procedemos a la eliminación de dichos registros.

Luego, con un .shape se observa cuantas filas y columnas nos quedaron luego del .dropna

In [3]:
df_g = df_g.dropna(how='all').reset_index(drop=True)
df_g.shape

(32135, 13)

<br>
<br>

#### Se chequean la cantidad de valores nulos por cada columna

In [4]:
df_g.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

<br>
<br>

#### Se verifican si hay registros de juegos duplicados, emparejando por el id del juego que debería ser un valor único

In [5]:
filas_duplicadas = df_g[df_g.duplicated(subset="id")]
filas_duplicadas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


<br>

Existen dos filas duplicadas, pero una de ellas contiene valor Nan en el id

In [6]:
filas_duplicadas = df_g[df_g.duplicated(subset="id", keep=False)]
filas_duplicadas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


<br>

Uno de ellos solo tiene un campo completo, el campo precio, y el otro tiene título del juego, así que, se decide filtrar por el campo título para ver si se consigue información relevante.

Luego de intentar aplicar un filtro en la columna 'title' sin éxito por contener valores Nan, se procede con otra columna

In [7]:
df_g[df_g['developer']=='Rocksteady Studios,Feral Interactive (Mac)']


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,http://store.steampowered.com/app/35140/Batman...,2010-03-26,"[Action, Batman, Stealth, Adventure, Third Per...",http://steamcommunity.com/app/35140/reviews/?b...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


<br>
<br>

Con la columna 'developer' se ve que el juego ya existe, con el id 200260, por lo que se procede a la eliminación de ambos registros encontrados como duplicados con id Nan, y se vuelve a verificar la existencia de duplicados (se espera que no arroje resultados)

In [8]:
df_g = df_g.drop([13894, 74, 30961])
filas_duplicadas = df_g[df_g.duplicated(subset="id", keep=False)]
filas_duplicadas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


<br>
<br>

## Se procede con la preparación del Dataframe para la eliminación de las columnas que no serán tenidas en cuenta para las consultas de la API, ni para nuestro modelo de recomendación.

<br>

#### Se rellenan los valores nulos de la columna 'title' con los valores que si existan en 'app_name'.

In [9]:
df_g['title'].fillna(df_g['app_name'], inplace=True)
df_g['title'].isna().sum()

C:\Users\Cebol\AppData\Local\Temp\ipykernel_21960\1275773724.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_g['title'].fillna(df_g['app_name'], inplace=True)


1

<br>
<br>

Partimos con 2050 valores nulos en 'title', ahora solo hay 1 valor nulo. Se procede a modificar ese dato nulo. (Dejamos lista la columna 'app_name' para su eliminación)

In [10]:
df_g['title'].fillna('Sin Datos', inplace=True)
df_g['title'].isna().sum()

0

<br>
<br>

Ahora procedemos a un tratamiento similar con el llenado de los valores nulos de la columna 'developer', obteniendo los mismos de la columna 'publisher', pero solo si estos valores existian previamente en la columna 'developer'(Para no crear nuevos valores de developer).

In [11]:
df_g['developer'] = df_g['developer'].str.lower()
df_g['publisher'] = df_g['publisher'].str.lower()
df_g['developer'].fillna(df_g['publisher'][df_g['publisher'].isin(df_g['developer'])], inplace=True)
df_g['developer'].isna().sum()

C:\Users\Cebol\AppData\Local\Temp\ipykernel_21960\2564882504.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_g['developer'].fillna(df_g['publisher'][df_g['publisher'].isin(df_g['developer'])], inplace=True)


3253

In [12]:
df_g['developer'].fillna('Sin Datos', inplace=True)
df_g['developer'].isna().sum()

0

<br>
<br>

Ahora si, se procede a la eliminación del dataframe de las columnas que no voy a tener en cuenta para la realización de la API, ni para el modelo de recomendación.

In [13]:
df_g = df_g.drop(['tags', 'specs', 'url', 'reviews_url', 'early_access', 'publisher', 'app_name'], axis=1)
df_g.columns

Index(['genres', 'title', 'release_date', 'price', 'id', 'developer'], dtype='object')

<br>
<br>

### Habiendo eliminado las columnas innecesarias, procedemos a la Transformación del resto de las columnas.

<br>

Se corrigen los duplicados en la columna 'title' por diferencias entre Mayúsculas y Minúsculas.

In [14]:
df_g['title'] = df_g['title'].str.lower()
df_g['title'].unique()

array(['lost summoner kitty', 'ironbound', 'real pool 3d - poolians', ...,
       'russian roads', 'exit 2 - directions', 'maze run vr'],
      dtype=object)

<br>
<br>

Para los Endpoint de la API, necesito el valor del año, por lo tanto, se procede a crear una columna nueva, tomando solo el valor del año, y a eliminar la columna 'release_date'

In [15]:
# Función para obtener la fecha
def obtener_anio(fecha):
    if pd.notna(fecha):
        if re.match(r'^\d{4}-\d{2}-\d{2}$', fecha):
            return fecha.split('-')[0]
    return 'Sin_dato_fecha'

# Crea columna nueva con el año
df_g['release_year'] = df_g['release_date'].apply(obtener_anio)


# elimina la columna 'release_date'
df_g = df_g.drop('release_date', axis=1)
df_g['release_year'].unique()

array(['2018', '2017', 'Sin_dato_fecha', '1997', '1998', '2016', '2006',
       '2005', '2003', '2007', '2002', '2000', '1995', '1996', '1994',
       '2001', '1993', '2004', '1999', '2008', '2009', '1992', '1989',
       '2010', '2011', '2013', '2012', '2014', '1983', '1984', '2015',
       '1990', '1988', '1991', '1985', '1982', '1987', '1981', '1986',
       '2021', '2019', '1975', '1970', '1980'], dtype=object)

<br>
<br>

Se necesita que la columna precio NO sea de tipo texto, por lo que a los Nan o los campos tipo texto "free to play", se decide convertirlos a 0, para poder trabajar esta columna y hacer sumas, etc.

In [16]:
def flotante(value):
    if pd.isna(value):
        return 0.0
    try:
        float_value = float(value)
        return float_value
    except:
        return 0.0

df_g['price'] = df_g['price'].apply(flotante)
df_g['price'] = df_g['price'].apply(lambda x: round(x, 2))
df_g['price'].dtype

dtype('float64')

<br>
<br>

De la columna 'genres' necesito obtener cada valor de la lista de los distintos géneros. Realizamos una transformación para obtener una columna de cada género. 

In [17]:
df_g = df_g.explode('genres')
df_g = df_g.dropna(subset=['genres'])
df_g.head(10)

,genres,title,price,id,developer,release_year
0,Action,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Casual,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Indie,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Simulation,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Strategy,lost summoner kitty,4.99,761140,kotoshiro,2018
1,Free to Play,ironbound,0.00,643980,secret level srl,2018
1,Indie,ironbound,0.00,643980,secret level srl,2018
1,RPG,ironbound,0.00,643980,secret level srl,2018
1,Strategy,ironbound,0.00,643980,secret level srl,2018
2,Casual,real pool 3d - poolians,0.00,670290,poolians.com,2017


<br>
<br>

Se renombra la columna 'id' 

In [18]:
df_g = df_g.rename(columns={'id':'item_id'})

<br>
<br>

El dataset quedaría conformado de la siguiente manera

In [19]:
df_g

,genres,title,price,item_id,developer,release_year
0,Action,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Casual,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Indie,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Simulation,lost summoner kitty,4.99,761140,kotoshiro,2018
0,Strategy,lost summoner kitty,4.99,761140,kotoshiro,2018
...,...,...,...,...,...,...
32132,Indie,russian roads,1.99,610660,laush dmitriy sergeevich,2018
32132,Racing,russian roads,1.99,610660,laush dmitriy sergeevich,2018
32132,Simulation,russian roads,1.99,610660,laush dmitriy sergeevich,2018
32133,Casual,exit 2 - directions,4.99,658870,"xropi,stev3ns",2017


In [20]:
mi_dict = {"Campo": [], "Tipo_de_dato": [], "Nulos": []}

for columna in df_g.columns:
    mi_dict["Campo"].append(columna)
    mi_dict["Tipo_de_dato"].append(df_g[columna].apply(type).unique())
    mi_dict["Nulos"].append(df_g[columna].isnull().sum())

df_info = pd.DataFrame(mi_dict)
df_info

,Campo,Tipo_de_dato,Nulos
0,genres,[<class 'str'>],0
1,title,[<class 'str'>],0
2,price,[<class 'float'>],0
3,item_id,[<class 'str'>],0
4,developer,[<class 'str'>],0
5,release_year,[<class 'str'>],0


<br>

#### Se carga del Dataset limpio

In [21]:
df_g.to_csv(r'C:\Users\Cebol\OneDrive\Escritorio\Datasets\Datasets\steam_games_limpio.csv', index=False, encoding='utf-8')